In [2]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

In [6]:
stock = pd.read_csv('C:/Users/user/Desktop/samsung_stock_volume.csv')
stock

,date,end,volume
0,2020.10.28,"58,900","10,669,385"
1,2020.10.27,"59,800","17,260,416"
2,2020.10.26,"60,400","13,268,226"
3,2020.10.23,"60,200","9,322,232"
4,2020.10.22,"60,100","14,294,095"
...,...,...,...
4285,2003.07.01,"362,000","260,070"
4286,2003.06.30,"355,000","312,166"
4287,2003.06.27,"361,000","455,494"
4288,2003.06.26,"364,500","345,345"


In [84]:
start_date = datetime.strptime('20030625', '%Y%m%d')
end_date = datetime.strptime('20201025', '%Y%m%d')

str_date_list = []
while start_date.strftime('%Y%m%d') != end_date.strftime('%Y%m%d'): # start_date가 end_date랑 같아지기 전까지 1씩 늘어나면서 리스트에 추가
    str_date_list.append(start_date.strftime('%Y.%m.%d'))
    start_date += timedelta(days=1)
str_date_list.append(end_date.strftime('%Y.%m.%d')) # end_date 포함안됐으므로 따로 추가

date_list=np.array([str_date_list])
all_date  = pd.DataFrame(date_list.T, columns=['date'])

all_date['date'] = pd.to_datetime(all_date['date'], format="%Y.%m.%d")
stock['date'] = pd.to_datetime(stock['date'], format="%Y.%m.%d")

df = pd.merge(all_date,stock, on='date', how='outer') # 기존주가데이터와 전체날짜를 date기준으로 outer 조인
df

,date,end,volume
0,2003-06-25,"359,500","345,721"
1,2003-06-26,"364,500","345,345"
2,2003-06-27,"361,000","455,494"
3,2003-06-28,NaN,NaN
4,2003-06-29,NaN,NaN
...,...,...,...
6331,2020-10-24,NaN,NaN
6332,2020-10-25,NaN,NaN
6333,2020-10-28,"58,900","10,669,385"
6334,2020-10-27,"59,800","17,260,416"


In [85]:
df['date'] = pd.to_datetime(df['date'], format="%Y-%m-%d")

In [79]:
df.to_csv('C:/Users/user/Desktop/before_interpolate.csv', index=False)

In [86]:
df['end'] = df['end'].str.replace(r',', '')
df['volume'] = df['volume'].str.replace(r',', '')
df

,date,end,volume
0,2003-06-25,359500,345721
1,2003-06-26,364500,345345
2,2003-06-27,361000,455494
3,2003-06-28,NaN,NaN
4,2003-06-29,NaN,NaN
...,...,...,...
6331,2020-10-24,NaN,NaN
6332,2020-10-25,NaN,NaN
6333,2020-10-28,58900,10669385
6334,2020-10-27,59800,17260416


In [87]:
df['end']=df['end'].astype(float)
df['volume']=df['volume'].astype(float)

In [104]:
df_stock = df.set_index('date')
df_stock['end'] = df_stock['end'].interpolate(method='time')
df_stock['volume'] = df_stock['volume'].interpolate(method='time')
df_stock

,end,volume
date,,
2003-06-25,359500.000000,3.457210e+05
2003-06-26,364500.000000,3.453450e+05
2003-06-27,361000.000000,4.554940e+05
2003-06-28,359000.000000,4.077180e+05
2003-06-29,357000.000000,3.599420e+05
...,...,...
2020-10-24,60266.666667,1.063756e+07
2020-10-25,60333.333333,1.195289e+07
2020-10-28,58900.000000,1.066938e+07


In [92]:
df_stock.to_csv('C:/Users/user/Desktop/after_interpolate.csv')

In [105]:
df_stock['min_in100'] = ''

In [106]:
for i in range(100, len(df['date'])):
    df_stock['min_in100'][i]=df_stock['volume'][i-100:i].min()
df_stock

<ipython-input-106-6537e451c0e5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stock['min_in100'][i]=df_stock['volume'][i-100:i].min()


,end,volume,min_in100
date,,,
2003-06-25,359500.000000,3.457210e+05,
2003-06-26,364500.000000,3.453450e+05,
2003-06-27,361000.000000,4.554940e+05,
2003-06-28,359000.000000,4.077180e+05,
2003-06-29,357000.000000,3.599420e+05,
...,...,...,...
2020-10-24,60266.666667,1.063756e+07,9.32223e+06
2020-10-25,60333.333333,1.195289e+07,9.32223e+06
2020-10-28,58900.000000,1.066938e+07,9.32223e+06


In [107]:
df_stock['d_score'] = ''

In [108]:
for i in range(100, len(df['date'])):
    df_stock['d_score'][i] = df_stock['volume'][i]-df_stock['min_in100'][i]
df_stock

<ipython-input-108-ffb837e15ab8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stock['d_score'][i] = df_stock['volume'][i]-df_stock['min_in100'][i]


,end,volume,min_in100,d_score
date,,,,
2003-06-25,359500.000000,3.457210e+05,,
2003-06-26,364500.000000,3.453450e+05,,
2003-06-27,361000.000000,4.554940e+05,,
2003-06-28,359000.000000,4.077180e+05,,
2003-06-29,357000.000000,3.599420e+05,,
...,...,...,...,...
2020-10-24,60266.666667,1.063756e+07,9.32223e+06,1.31533e+06
2020-10-25,60333.333333,1.195289e+07,9.32223e+06,2.63066e+06
2020-10-28,58900.000000,1.066938e+07,9.32223e+06,1.34715e+06


In [109]:
df_stock.to_csv('C:/Users/user/Desktop/d_score.csv', index=False)